In [1]:
import numpy as np
import scipy.spatial.distance as distance
import mdtraj as md
import sys

In [2]:
%load_ext autoreload

In [3]:
sys.path.append("/home/gmancini/Dropbox/appunti/Clustering/src/")
import myclusters, mymetrics
sys.path.append("/home/gmancini/devel/ML/sanz/HVL")
%autoreload 2

**Test wss vs inertia**

In [4]:
X = np.random.rand(100,3)
d = distance.pdist(X)
D = distance.squareform(d)
D.shape

(100, 100)

In [5]:
np.max(D-D.T)

0.0

In [6]:
my_estimator = myclusters.PAM(D=D,K=2,niter=500,nrun=10,boot='kmeans++',conv=1e-5,metric='precomputed')
my_estimator.do_clustering()
my_estimator.clusters, my_estimator.inertia

(array([59, 59, 59, 10, 59, 59, 10, 59, 59, 59, 10, 59, 59, 10, 59, 59, 59,
        59, 10, 59, 59, 10, 59, 10, 10, 59, 59, 59, 59, 59, 59, 59, 59, 59,
        10, 10, 59, 59, 59, 10, 59, 59, 59, 59, 59, 10, 59, 59, 59, 59, 59,
        59, 59, 59, 59, 10, 59, 59, 59, 59, 10, 10, 10, 10, 59, 59, 59, 59,
        59, 59, 59, 59, 59, 10, 59, 59, 59, 10, 59, 59, 59, 59, 59, 59, 59,
        59, 10, 59, 10, 10, 59, 10, 10, 59, 59, 59, 59, 10, 59, 59]),
 22.99916620198309)

In [7]:
myeval = mymetrics.cluster_eval(D=D,metric='precomputed',clusters=my_estimator.clusters)
psf,wss = myeval(method='psF')
psf, wss, wss-my_estimator.inertia

(47.10064730382948, 22.99916620198309, 0.0)

**Test RMSD is symmetric**

In [8]:
traj = md.load("dftba/traj_hvl_dftba.pdb")
traj

<mdtraj.Trajectory with 2291 frames, 52 atoms, 1 residues, without unitcells at 0x7ff3393cf320>

In [9]:
rmsds = md.rmsd(traj, traj)
rmsds.shape

(2291,)

In [10]:
index=list(range(20))
index

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [11]:
RMSD = np.empty((2291,2291))
for i in range(2291):
    RMSD[i,i:] = md.rmsd(traj[i:],traj,i,precentered=False,atom_indices=index)
RMSD.shape

(2291, 2291)

In [12]:
RMSD[np.tril_indices(2291)] = RMSD[np.triu_indices(2291)]

In [13]:
np.nonzero(RMSD-RMSD.T)

(array([   0,    0,    0, ..., 2290, 2290, 2290]),
 array([   2,    3,    4, ..., 2286, 2287, 2288]))

In [14]:
np.max(RMSD-RMSD.T)

0.32002362608909607

In [15]:
a = md.rmsd(traj,traj,10,precentered=False,atom_indices=index)
print(a)

[0.14171752 0.14171803 0.1724065  ... 0.1724298  0.17242268 0.11894155]


In [16]:
np.where(a==0)

(array([  10,   48,  112,  206,  320,  441,  471,  478,  503,  553,  558,
         643,  671,  733,  882, 1116, 1455, 1463, 1743, 1915, 1929, 2009,
        2086, 2166, 2178, 2219, 2252]),)

In [17]:
a = md.rmsd(traj,traj,48,precentered=False,atom_indices=index)
print(a)

[0.14172569 0.1417262  0.17241023 ... 0.17243436 0.17242697 0.11894806]


In [18]:
np.where(a==0)

(array([  10,   48,  112,  206,  320,  441,  471,  478,  503,  553,  558,
         643,  671,  882, 1116, 1455, 1463, 1743, 1915, 1929, 2009, 2166,
        2178, 2219, 2252]),)

In [19]:
md.rmsd(traj[48],traj,10,precentered=False,atom_indices=index)

array([0.], dtype=float32)

In [20]:
md.rmsd(traj[10],traj,48,precentered=False,atom_indices=index)

array([0.], dtype=float32)

In [42]:
def myrmsd(X1,X2,mask=None):
    #Kabsch algorithm
    #see https://cnx.org/contents/HV-RsdwL@23/Molecular-Distance-Measures
    if mask is None:
        mask = list(range(X1.shape[0]))
    assert X1[mask].shape == X2[mask].shape
    nat = len(mask)
    Com1 = np.mean(X1[mask],axis=0)
    Com2 = np.mean(X2[mask],axis=0)
    C1 = X1[mask]-Com1
    C2 = X2[mask]-Com2
    Cov = np.dot(C1.T, C2)
    V, S, W = np.linalg.svd(Cov)
    d = np.sign(np.linalg.det(Cov))
    D = np.eye(3)
    D[2,2] = d
    R = np.dot(V,np.dot(D,W))
    rotC2 = np.dot(C2, R)
    displ = np.mean(C1-rotC2,axis=0)
    rotC2 = rotC2 -displ
    dim = C2.shape[1]
    rmsd = 0.
    for v, w in zip(C1, rotC2):
        rmsd = sum([(v[i] - w[i])**2.0 for i in range(dim)])
    return np.sqrt(rmsd/nat)

In [43]:
myrmsd(traj[10].xyz[0],traj[48].xyz[0])

3.334499765894962e-07

In [44]:
traj[10].save_pdb("test10.pdb")
traj[48].save_pdb("test48.pdb")

In [45]:
RMSD = np.empty((2291,2291))
for i in range(2291):
    for j in range(2291):
        RMSD[i,j] = myrmsd(traj[i].xyz[0],traj[j].xyz[0],mask=index)
RMSD.shape

(2291, 2291)